In [1]:
import pandas as pd

from sprime_mapping import map_sprime_segments
from sprime_info import compute_sprime_info

/home/jalh/anaconda3/envs/gaishi-analysis150126/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# these two files are necessary: score_file with SPrime output and original vcf_file output
vcf_file = "archie.1_biallelic.vcf.gz"
score_file = "sprime.1src.out.100000.score"


In [3]:
# for getting only the introgressed tracts in the target individuals, we also have to provide a list with the individual names (have to correspond to the vcf-file) we are interested in
# if not provided, for all individuals in the vcf-file the introgression status is checked (including reference individuals etc. - this is usually not desired / reasonable, but perhaps one want to check)
tgt_individuals_file = "archie.1.tgt.ind.list"

In [4]:
# if we ant to write the file, also an out_file has to be scpecified
out_file_wo_filtering = "archie.0.sprime.100000.inferred_wo_filtering.bed"
out_file_every_snp = "archie.0.sprime.100000.inferred_every_snp.bed"
out_file_frac_50 = "archie.0.sprime.100000.inferred_fraction_050.bed"




In [42]:
# open file with tgt individuals

In [5]:
with open(tgt_individuals_file) as f:
    tgt_individuals = [line.strip() for line in f if line.strip()]


In [6]:
tgt_individuals

['tsk_50',
 'tsk_51',
 'tsk_52',
 'tsk_53',
 'tsk_54',
 'tsk_55',
 'tsk_56',
 'tsk_57',
 'tsk_58',
 'tsk_59']

In [7]:
### Let's also check the sprime output file

In [ ]:
df = pd.read_csv(score_file, sep="\t")

# Count the number of SNPs per SEGMENT
snp_counts = df['SEGMENT'].value_counts()

snp_counts_df = snp_counts.reset_index()
snp_counts_df.columns = ['SEGMENT', 'SNP_COUNT']

print(snp_counts_df)
print(snp_counts_df["SNP_COUNT"].sum())

    SEGMENT  SNP_COUNT
0         0         75
1         1         47
2         4         42
3         5         36
4         2         31
5         3         30
6        11         29
7        10         28
8         6         27
9         7         24
10        9         23
11        8         22
12       13         21
13       12         20
455


### In this run of the function, we create an output-file (bed-file)
## We do not apply any filtering (see below for possible filtering steps)
## With these settings, the first and last SNP, which matches the archaic SNPs of the segment, is used as border of the segment

In [9]:
df = map_sprime_segments(
    score_file,
    vcf_file,
    out_file_wo_filtering, only_tract_output=True, phased=True, target_individuals=tgt_individuals, return_full_records=False)

### we can also show the full records, including the nsnps in each segment, the fraction of archaic snps in the segment for a specific individual and the snps present
### out_file is from now set to None, so no new out_file is created

In [10]:
df = map_sprime_segments(
    score_file,
    vcf_file,
    None, only_tract_output=True, phased=True, target_individuals=tgt_individuals, return_full_records=True)

In [11]:
df.head()


,chrom,start,end,individual,haplotype,nsnps,segment_fraction,segment,all_snps_present,archaic_snps_present
0,1,238504,314341,tsk_50,1,28,1.000000,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802..."
1,1,238504,267892,tsk_52,1,7,0.250000,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802..."
2,1,238504,279660,tsk_53,1,9,0.321429,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802..."
3,1,238504,314341,tsk_57,1,28,1.000000,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802..."
4,1,267891,314341,tsk_58,2,22,0.785714,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[267891, 268554, 279659, 281915, 290557, 29327..."


In [12]:
df.tail()

,chrom,start,end,individual,haplotype,nsnps,segment_fraction,segment,all_snps_present,archaic_snps_present
52,1,21560934,21560935,tsk_57,2,1,0.021277,1,"[21541283, 21541302, 21542755, 21544262, 21544...",[21560934]
53,1,21560934,21560935,tsk_59,1,1,0.021277,1,"[21541283, 21541302, 21542755, 21544262, 21544...",[21560934]
54,1,21560934,21560935,tsk_59,2,1,0.021277,1,"[21541283, 21541302, 21542755, 21544262, 21544...",[21560934]
55,1,21905012,22041832,tsk_56,1,24,0.800000,3,"[21905012, 21906798, 21911405, 21915067, 21916...","[21905012, 21906798, 21911405, 21915067, 21916..."
56,1,21927036,22028446,tsk_56,2,6,0.200000,3,"[21905012, 21906798, 21911405, 21915067, 21916...","[21927036, 21932237, 21958351, 21991539, 22000..."


# Computation of averages

### We now additionally compute the length of the regions as well as the segments per individual
## afterwards, we can average over these values and get the desired results

In [13]:
res_wo_filtering = compute_sprime_info(df)

In [14]:
# the df now also has interval length
res_wo_filtering["df"].head()

,chrom,start,end,individual,haplotype,nsnps,segment_fraction,segment,all_snps_present,archaic_snps_present,individual_haplotype,interval_length
0,1,238504,314341,tsk_50,1,28,1.000000,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802...",tsk_50_1,75837
1,1,238504,267892,tsk_52,1,7,0.250000,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802...",tsk_52_1,29388
2,1,238504,279660,tsk_53,1,9,0.321429,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802...",tsk_53_1,41156
3,1,238504,314341,tsk_57,1,28,1.000000,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[238504, 251513, 255766, 257368, 257723, 25802...",tsk_57_1,75837
4,1,267891,314341,tsk_58,2,22,0.785714,10,"[238504, 251513, 255766, 257368, 257723, 25802...","[267891, 268554, 279659, 281915, 290557, 29327...",tsk_58_2,46450


In [15]:
# total length of introgressed regions per individual/haplotype
res_wo_filtering["total_length_ind_hap"]

,individual_haplotype,total_length,individual,haplotype
0,tsk_50_1,192915,tsk_50,1
1,tsk_50_2,118567,tsk_50,2
2,tsk_51_1,45941,tsk_51,1
3,tsk_51_2,92132,tsk_51,2
4,tsk_52_1,29389,tsk_52,1
5,tsk_52_2,199510,tsk_52,2
6,tsk_53_1,178030,tsk_53,1
7,tsk_53_2,178958,tsk_53,2
8,tsk_54_1,13709,tsk_54,1
9,tsk_54_2,107553,tsk_54,2


In [16]:
# total length of introgressed regions per individual/haplotype and segment
res_wo_filtering["total_length_ind_hap_seg"].iloc[0:5]

,individual_haplotype,segment,total_length,individual,haplotype
0,tsk_50_1,1,117078,tsk_50,1
1,tsk_50_1,10,75837,tsk_50,1
2,tsk_50_2,0,1,tsk_50,2
3,tsk_50_2,1,72626,tsk_50,2
4,tsk_50_2,13,45940,tsk_50,2


In [17]:
#segments per individual/haplotype 
res_wo_filtering["segments_per_ind"]

,individual_haplotype,segments,nr_segments,individual,haplotype
0,tsk_50_1,"[1, 10]",2,tsk_50,1
1,tsk_50_2,"[0, 1, 13]",3,tsk_50,2
2,tsk_51_1,"[1, 13]",2,tsk_51,1
3,tsk_51_2,"[0, 1]",2,tsk_51,2
4,tsk_52_1,"[1, 10]",2,tsk_52,1
5,tsk_52_2,"[1, 4, 5]",3,tsk_52,2
6,tsk_53_1,"[0, 1, 6, 10]",4,tsk_53,1
7,tsk_53_2,"[1, 4]",2,tsk_53,2
8,tsk_54_1,"[1, 4]",2,tsk_54,1
9,tsk_54_2,"[1, 6]",2,tsk_54,2


## Average number of segments per individual

In [18]:
avg_per_individual = (
    res_wo_filtering["segments_per_ind"].groupby("individual")["nr_segments"]
      .mean()
      .reset_index(name="avg_nr_segments")
)


In [19]:
avg_per_individual

,individual,avg_nr_segments
0,tsk_50,2.5
1,tsk_51,2.0
2,tsk_52,2.5
3,tsk_53,3.0
4,tsk_54,2.0
5,tsk_55,2.0
6,tsk_56,3.5
7,tsk_57,3.5
8,tsk_58,3.5
9,tsk_59,4.0


## or per chromosome, ,i.e. individual haplotypes

In [20]:
avg_per_individual_and_haplotype = (
    res_wo_filtering["segments_per_ind"].groupby(["individual", "haplotype"])["nr_segments"]
      .mean()
      .reset_index(name="avg_nr_segments")
)

In [21]:
avg_per_individual_and_haplotype

,individual,haplotype,avg_nr_segments
0,tsk_50,1,2.0
1,tsk_50,2,3.0
2,tsk_51,1,2.0
3,tsk_51,2,2.0
4,tsk_52,1,2.0
5,tsk_52,2,3.0
6,tsk_53,1,4.0
7,tsk_53,2,2.0
8,tsk_54,1,2.0
9,tsk_54,2,2.0


In [22]:
#Filtering

### we can specify minimum fractions for the amount of archaic snps which has to be present in an individual
### now get fewer hits (in particular, one can also set a threshold afterwards, because the fraction is stored in segment_faction)

In [23]:
df_50 = map_sprime_segments(
    score_file,
    vcf_file,
    out_file_frac_50, only_tract_output=True, phased=True, return_full_records=False, target_individuals=tgt_individuals,
    segment_fraction=0.5)

In [24]:
df_50_full = map_sprime_segments(
    score_file,
    vcf_file,
    None, only_tract_output=True, phased=True, return_full_records=True, target_individuals=tgt_individuals,
    segment_fraction=0.5)

In [25]:
# now the number of fragments in the dataframe is considerably smaller 
print(len(df_50_full))

24


In [26]:
df_50_full.tail()

,chrom,start,end,individual,haplotype,nsnps,segment_fraction,segment,all_snps_present,archaic_snps_present
19,1,20647984,20693924,tsk_50,2,21,1.000000,13,"[20647984, 20648821, 20649517, 20649859, 20650...","[20647984, 20648821, 20649517, 20649859, 20650..."
20,1,20647984,20693924,tsk_51,1,21,1.000000,13,"[20647984, 20648821, 20649517, 20649859, 20650...","[20647984, 20648821, 20649517, 20649859, 20650..."
21,1,21541283,21572805,tsk_58,1,27,0.574468,1,"[21541283, 21541302, 21542755, 21544262, 21544...","[21541283, 21541302, 21542755, 21544262, 21544..."
22,1,21557378,21674456,tsk_50,1,33,0.702128,1,"[21541283, 21541302, 21542755, 21544262, 21544...","[21557378, 21559072, 21560497, 21560541, 21560..."
23,1,21905012,22041832,tsk_56,1,24,0.800000,3,"[21905012, 21906798, 21911405, 21915067, 21916...","[21905012, 21906798, 21911405, 21915067, 21916..."


In [27]:
res_frac50 = compute_sprime_info(df_50_full)



In [28]:
# total length of introgressed regions per individual/haplotype
res_frac50["total_length_ind_hap"]


,individual_haplotype,total_length,individual,haplotype
0,tsk_50_1,192915,tsk_50,1
1,tsk_50_2,45940,tsk_50,2
2,tsk_51_1,45940,tsk_51,1
3,tsk_52_2,199509,tsk_52,2
4,tsk_53_1,107552,tsk_53,1
5,tsk_53_2,178957,tsk_53,2
6,tsk_54_2,107552,tsk_54,2
7,tsk_55_2,107552,tsk_55,2
8,tsk_56_1,420796,tsk_56,1
9,tsk_56_2,71084,tsk_56,2


In [29]:
# total length of introgressed regions per individual/haplotype and segment
res_frac50["total_length_ind_hap_seg"]

,individual_haplotype,segment,total_length,individual,haplotype
0,tsk_50_1,1,117078,tsk_50,1
1,tsk_50_1,10,75837,tsk_50,1
2,tsk_50_2,13,45940,tsk_50,2
3,tsk_51_1,13,45940,tsk_51,1
4,tsk_52_2,4,141874,tsk_52,2
5,tsk_52_2,5,57635,tsk_52,2
6,tsk_53_1,6,107552,tsk_53,1
7,tsk_53_2,4,178957,tsk_53,2
8,tsk_54_2,6,107552,tsk_54,2
9,tsk_55_2,6,107552,tsk_55,2


In [30]:
#segments per individual/haplotype 
res_frac50["segments_per_ind"]

,individual_haplotype,segments,nr_segments,individual,haplotype
0,tsk_50_1,"[1, 10]",2,tsk_50,1
1,tsk_50_2,[13],1,tsk_50,2
2,tsk_51_1,[13],1,tsk_51,1
3,tsk_52_2,"[4, 5]",2,tsk_52,2
4,tsk_53_1,[6],1,tsk_53,1
5,tsk_53_2,[4],1,tsk_53,2
6,tsk_54_2,[6],1,tsk_54,2
7,tsk_55_2,[6],1,tsk_55,2
8,tsk_56_1,"[0, 3]",2,tsk_56,1
9,tsk_56_2,[11],1,tsk_56,2


### if we addionally specify min_snps, we get more fragments, because within the segment, fragments of contiguous archaic SNPs with a length of at least 3 are determined and these fragments are added to the final list (whereas shorter fragments are discarded)

In [31]:
dff_frac50_minsnps3 = map_sprime_segments(
    score_file,
    vcf_file,
    None, only_tract_output=True, phased=True, return_full_records=True, target_individuals=tgt_individuals,
    segment_fraction=0.5, min_snps=3)

In [32]:
# now the number of fragments in the dataframe is considerably smaller 
print(len(dff_frac50_minsnps3))

37


In [33]:
dff_frac50_minsnps3.tail()

,chrom,start,end,individual,haplotype,nsnps,segment_fraction,segment,archaic_snps_present
32,1,21905012,21923919,tsk_56,1,6,0.8,3,"[21905012, 21906798, 21911405, 21915067, 21916..."
33,1,21935944,21951411,tsk_56,1,5,0.8,3,"[21935944, 21938125, 21949125, 21950785, 21951..."
34,1,21964563,21978154,tsk_56,1,5,0.8,3,"[21964563, 21966130, 21970779, 21977944, 21978..."
35,1,21994331,21998049,tsk_56,1,3,0.8,3,"[21994331, 21995907, 21998048]"
36,1,22033401,22041832,tsk_56,1,4,0.8,3,"[22033401, 22036294, 22038925, 22041831]"


### Finally, by setting single_snp_bed=True, we can map every single SNPs (so also in the bed-file there are exclusively entries of length 1, one entry for each SNP, no region)

In [34]:
df_single_snp = map_sprime_segments(
    score_file,
    vcf_file,
    out_file_every_snp, only_tract_output=True, phased=True, return_full_records=False, target_individuals=tgt_individuals,
    single_snp_bed=True)

In [35]:
df_single_snp = map_sprime_segments(
    score_file,
    vcf_file,
    None, only_tract_output=True, phased=True, return_full_records=True, target_individuals=tgt_individuals,
    single_snp_bed=True)

In [36]:
df_single_snp

,chrom,start,end,individual,haplotype,nsnps,segment_fraction,segment,archaic_snps_present
0,1,238504,238505,tsk_50,1,1,0.035714,10,[238504]
1,1,251513,251514,tsk_50,1,1,0.035714,10,[251513]
2,1,255766,255767,tsk_50,1,1,0.035714,10,[255766]
3,1,257368,257369,tsk_50,1,1,0.035714,10,[257368]
4,1,257723,257724,tsk_50,1,1,0.035714,10,[257723]
...,...,...,...,...,...,...,...,...,...
866,1,21932237,21932238,tsk_56,2,1,0.033333,3,[21932237]
867,1,21958351,21958352,tsk_56,2,1,0.033333,3,[21958351]
868,1,21991539,21991540,tsk_56,2,1,0.033333,3,[21991539]
869,1,22000748,22000749,tsk_56,2,1,0.033333,3,[22000748]


### in this case, the function gives us directly the number of archaic SNPs per individual/haplotype

In [37]:
archaic_snp_counts_per_individual = df_single_snp['individual'].value_counts()
print(archaic_snp_counts_per_individual)

individual
tsk_57    140
tsk_56    136
tsk_59    110
tsk_58     92
tsk_53     86
tsk_50     85
tsk_52     83
tsk_51     57
tsk_54     43
tsk_55     39
Name: count, dtype: int64


### and we can also easy calcualte the avgerage archaic SNPs per individual


In [38]:
archaic_snp_counts_per_individual.mean()

87.1

In [39]:
archaic_snp_counts_per_individual_haplotype = df_single_snp.value_counts(['individual', 'haplotype'])
print(archaic_snp_counts_per_individual_haplotype)

individual  haplotype
tsk_57      1            126
tsk_56      1             99
tsk_59      1             75
tsk_52      2             75
tsk_50      1             61
tsk_58      1             50
tsk_53      1             43
            2             43
tsk_58      2             42
tsk_56      2             37
tsk_59      2             35
tsk_51      2             35
tsk_55      2             28
tsk_54      2             28
tsk_50      2             24
tsk_51      1             22
tsk_54      1             15
tsk_57      2             14
tsk_55      1             11
tsk_52      1              8
Name: count, dtype: int64


In [40]:
archaic_snp_counts_per_individual_haplotype.mean()

43.55

In [41]:
archaic_snp_counts_per_individual_haplotype_segment = df_single_snp.value_counts(['individual', 'haplotype', 'segment'])
print(archaic_snp_counts_per_individual_haplotype_segment)

individual  haplotype  segment
tsk_56      1          0          74
tsk_57      1          0          74
tsk_53      2          4          42
tsk_52      2          4          38
                       5          36
tsk_51      2          0          34
tsk_50      1          1          33
tsk_59      1          4          30
tsk_56      2          11         29
tsk_57      1          10         28
tsk_50      1          10         28
tsk_54      2          6          27
tsk_58      1          1          27
tsk_53      1          6          27
tsk_55      2          6          27
tsk_59      1          5          24
tsk_56      1          3          24
tsk_57      1          7          23
tsk_59      2          9          22
tsk_58      2          10         22
tsk_51      1          13         21
tsk_50      2          13         21
tsk_59      1          2          19
tsk_54      1          4          14
tsk_57      2          4          12
tsk_59      2          2          12
tsk_58 